In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# 자산 리스트
asset_list = [
    'SPY', 'AGG', 'GLD', 'XLU', 'XLV',
    'TLT', 'IYR', 'TIP', 'BND', 'LQD'
]

# 데이터 수집
start_date = '2010-01-01'
end_date = '2023-10-01'
price_data = yf.download(asset_list, start=start_date, end=end_date)['Adj Close']
price_data = price_data.fillna(method='ffill').fillna(method='bfill')


In [ ]:
# 수익률 계산
returns = price_data.pct_change().dropna()

# 스케일링
scalers = {}
scaled_returns = pd.DataFrame(index=returns.index)

for asset in asset_list:
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(returns[[asset]])
    scaled_returns[asset] = scaled_data.flatten()
    scalers[asset] = scaler  # 나중에 역스케일링을 위해 저장


In [ ]:
def create_dataset(dataset, look_back=60):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back)])
        Y.append(dataset[i + look_back])
    return np.array(X), np.array(Y)

look_back = 60  # 과거 60일의 데이터를 사용

X_dict = {}
y_dict = {}

for asset in asset_list:
    data = scaled_returns[asset].values
    X, y = create_dataset(data, look_back)
    X_dict[asset] = X
    y_dict[asset] = y


In [ ]:
# 모델 학습을 위한 설정
batch_size = 32
epochs = 20

# 모델 저장을 위한 딕셔너리
models = {}

for asset in asset_list:
    X = X_dict[asset]
    y = y_dict[asset]
    
    # 데이터 분할 (80% 학습, 20% 검증)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # LSTM 입력 형태로 변환
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    # LSTM 모델 구축
    model = Sequential()
    model.add(LSTM(50, input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # 조기 종료 콜백 설정
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    
    # 모델 학습
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stop], verbose=0)
    
    # 모델 저장
    models[asset] = model


In [ ]:
predicted_returns = {}

for asset in asset_list:
    X = X_dict[asset]
    y = y_dict[asset]
    
    # 전체 데이터를 사용하여 예측 수행
    X_all = np.reshape(X, (X.shape[0], X.shape[1], 1))
    model = models[asset]
    predictions = model.predict(X_all)
    
    # 역스케일링
    scaler = scalers[asset]
    predictions = scaler.inverse_transform(predictions)
    actual = scaler.inverse_transform(y.reshape(-1, 1))
    
    # 마지막 예측 값 저장
    predicted_returns[asset] = predictions[-1][0]
    
    # 모델 평가 (원한다면)
    # mse = mean_squared_error(actual, predictions)
    # print(f"{asset} - MSE: {mse}")


In [ ]:
# 위험 허용도 설정 (0에 가까울수록 보수적)
risk_tolerance = 0.2

# 변동성 계산
volatility = returns.std()

# 예상 수익률 데이터프레임 생성
predicted_returns_df = pd.DataFrame({
    'predicted_return': predicted_returns,
    'volatility': volatility
})

# 위험 대비 수익률 계산
predicted_returns_df['adjusted_score'] = predicted_returns_df['predicted_return'] / predicted_returns_df['volatility'] * risk_tolerance

# 가중치 계산 및 정규화
predicted_returns_df['weight'] = predicted_returns_df['adjusted_score'] / predicted_returns_df['adjusted_score'].sum()

# 추천 포트폴리오 출력
recommended_portfolio = predicted_returns_df[['predicted_return', 'volatility', 'weight']]
print("추천 포트폴리오 구성:")
print(recommended_portfolio.sort_values('weight', ascending=False))


3. 추가적인 고려사항 및 개선 방안
3.1. 모델 성능 향상
하이퍼파라미터 튜닝: look_back, batch_size, epochs, num_heads, key_dim 등의 하이퍼파라미터를 조정하여 모델 성능을 향상시킬 수 있습니다.
교차 검증: 시계열 교차 검증 방법을 사용하여 모델의 일반화 성능을 평가할 수 있습니다.
피처 엔지니어링: 기술적 지표(MA, RSI 등)나 거시경제 지표를 추가하여 모델의 입력으로 사용하면 예측 성능이 향상될 수 있습니다.
3.2. 모델 비교 및 선택
LSTM과 Transformer 모델을 모두 적용하여 성능을 비교하고, 더 나은 모델을 선택할 수 있습니다.
모델 평가 지표로 MSE, MAE, MAPE 등을 사용하여 모델의 예측 정확도를 확인합니다.
3.3. 멀티 아웃풋 모델
각 자산별로 개별 모델을 학습하는 대신, 여러 자산의 수익률을 동시에 예측하는 멀티 아웃풋 모델을 구축할 수 있습니다.
이는 자산 간의 상관관계를 학습하는 데 도움이 될 수 있습니다.
3.4. 실시간 시스템 구축
모델을 주기적으로 재학습하거나 업데이트하여 최신 데이터를 반영할 수 있습니다.
새로운 데이터가 들어올 때마다 예측을 업데이트하여 포트폴리오를 동적으로 조정합니다.